In [2]:
#!pip install openai flaml

     |████████████████████████████████| 225 kB 4.2 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.5.0
    Uninstalling typing-extensions-4.5.0:
      Successfully uninstalled typing-extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autoviml 0.1.713 requires numpy~=1.19.5, but you have numpy 1.23.5 which is incompatible.
autoviml 0.1.713 requires xgboost~=1.1.1, but you have xgboost 1.7.5 which is incompatible.


In [11]:
import pandas as pd
import openai
from sklearn.model_selection import train_test_split
import os
import datetime
import flaml
import numpy as np
from sklearn.metrics import accuracy_score

In [17]:
df = pd.read_csv('~/fraudTrain.csv')

In [18]:
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('trans_num', axis=1)
df = df.drop('cc_num', axis=1)
df = df.drop('unix_time', axis=1)

In [19]:
mapping_dict = {1: 'Yes', 0: 'No'}
df['is_fraud'] = df['is_fraud'].map(mapping_dict)

In [20]:
mapping_dict = {'M': 'Male', 'F': 'Female'}
df['gender'] = df['gender'].map(mapping_dict)

In [21]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'], format='%Y-%m-%d %H:%M:%S')
df['dob'] = pd.to_datetime(df['dob'], format='%Y-%m-%d')

In [22]:
current_date = datetime.datetime.now().date()
df['age'] = current_date.year - df['dob'].dt.year

In [23]:
fraud_yes = df[df['is_fraud'] == 'Yes'].sample(n=500, random_state=42)
fraud_no = df[df['is_fraud'] == 'No'].sample(n=500, random_state=42)
merged_df = pd.concat([fraud_yes, fraud_no])
merged_df = merged_df.reset_index(drop=True)

In [9]:
df = merged_df.copy()

In [51]:
api_key = os.environ['OPENAI_API_KEY'] 
client = OpenAI(api_key=api_key)

# Load your JSONL data
file_path = 'manual_template.jsonl'  # Replace with your actual file path
df = pd.read_json(file_path, lines=True)

# Function to get embeddings from OpenAI
def get_embeddings(text):
    response = client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return response.data[0].embedding

# Convert 'output' to a binary classification target
df['is_fraud'] = df['output'].apply(lambda x: 1 if 'Yes' in x else 0)

# Get embeddings for the 'record' field
df['embeddings'] = df['record'].apply(get_embeddings)

KeyError: 'OPENAI_API_KEY'

In [ ]:
df.head()

In [ ]:
embedded_df = df[['embeddings', 'is_fraud']]

In [ ]:
X = np.stack(embedded_df['embeddings'])
y = embedded_df['is_fraud'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

automl = flaml.AutoML()
automl_settings = {
    "time_budget": 600,  # Maximum time in seconds for AutoML to run
    "task": "binary",    # Task type: binary classification
    "log_file_name": "automl.log",  # Optional log file for tracking results
}

# Search for the best model and hyperparameters
automl.fit(X_train, y_train, **automl_settings)

# Make predictions on the test data
y_pred = automl.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Optionally, you can access other results, such as the best model found:
print(automl.best_estimator)
print(automl.best_config)
print(automl.best_loss)